In [40]:
import ee
ee.Initialize()

In [41]:
# Define la fecha inicial y final en formato YYYY-MM-DD
start_date = '2020-01-01'
end_date = '2020-12-31'

# Cargar archivo shapefile
polygon = ee.FeatureCollection("projects/ee-bismarksr17/assets/LOTES_TEST_NUBES")

In [42]:
# Crear una colección de imágenes Sentinel-2
sentinel_2_col = ee.ImageCollection("COPERNICUS/S2").filterDate(start_date, end_date)
# Aplicar filtro de área
sentinel_2_col_area = sentinel_2_col.filterBounds(polygon)

In [43]:
# Función para calcular el porcentaje de nubosidad dentro del área del polígono
def cloud_percentage(image):
    cloud_mask = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud')
    cloud_area = cloud_mask.multiply(ee.Image.pixelArea()).reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= polygon.geometry(),
        scale= 10
    ).get('cloud')
    return ee.Number(cloud_area).divide(polygon.geometry().area()).multiply(100)

In [44]:
# Aplicar función para calcular el porcentaje de nubosidad
sentinel_2_col_cloud = sentinel_2_col_area.map(cloud_percentage)

In [45]:
# Filtra las imágenes con menos del 20% de nubosidad dentro del área del polígono
sentinel_2_col_filtered = sentinel_2_col_cloud.filter(ee.Filter.lt('cloud_percentage', 20))

In [46]:
type(sentinel_2_col_filtered)

ee.imagecollection.ImageCollection

In [47]:
import geemap

In [49]:
viz_img = {
    'bands':['RED','GREEN','BLUE'],
    'min':0.0,
    'max':0.5,
    'fillColorOpacity': '0'
}

In [50]:
# Crear un mapa vacío
map = geemap.Map()

# Agregar las imágenes del ImageCollection al mapa
map.addLayer(sentinel_2_col_filtered.first(), vis_params=viz_img, name='Sentinel-2')
#map.addLayer(polygon, {}, 'dd')
# Mostrar el mapa
map.show()

EEException: Collection.map: A mapped algorithm must return a Feature or Image.